In [1]:
!pip install transformers datasets torch scikit-learn tqdm numpy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatibl

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

teacher_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42) 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from torch.utils.data import DataLoader

batch_size = 16
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

In [5]:
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
import torch
from tqdm import tqdm

model_name = "textattack/bert-base-uncased-imdb"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name, num_labels=2)
teacher_model.to(device)

optimizer = AdamW(teacher_model.parameters(), lr=5e-5)
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
teacher_model.train()
for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
         batch = {k: v.to(device) for k, v in batch.items()}
         outputs = teacher_model(**batch)
         loss = outputs.loss
         loss.backward()
         optimizer.step()
         lr_scheduler.step()
         optimizer.zero_grad()
teacher_model.eval()
for param in teacher_model.parameters():
    param.requires_grad = False 

teacher_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
teacher_model.to(device)
teacher_model.eval()
for param in teacher_model.parameters():
     param.requires_grad = False

print("Öğretmen modeli hazırlandı (veya yüklendi).")
teacher_model.save_pretrained("./teacher_model")

2025-04-27 05:32:38.721244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745731958.938508      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745731958.999345      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1563/1563 [25:55<00:00,  1.01it/s]


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Öğretmen modeli hazırlandı (veya yüklendi).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
from transformers import BertConfig, BertForSequenceClassification # VEYA AutoModel...

student_config = BertConfig(
    num_hidden_layers=4,       # BERT-base'de 12'dir
    hidden_size=512,           # BERT-base'de 768'dir
    intermediate_size=2048,    # BERT-base'de 3072'dir
    num_attention_heads=8,     # BERT-base'de 12'dir
    max_position_embeddings=512,
    vocab_size=teacher_model.config.vocab_size, 
    num_labels=2              
)

student_model = BertForSequenceClassification(config=student_config)
student_model.to(device)

print(f"Öğrenci Modeli Parametre Sayısı: {student_model.num_parameters()}")
print(f"Öğretmen Modeli Parametre Sayısı: {teacher_model.num_parameters()}")

Öğrenci Modeli Parametre Sayısı: 28764674
Öğretmen Modeli Parametre Sayısı: 109483778


In [7]:
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_logits = teacher_logits / temperature
    soft_student_logits = student_logits / temperature

    soft_loss = F.kl_div(
        F.log_softmax(soft_student_logits, dim=-1),
        F.softmax(soft_teacher_logits, dim=-1),
        reduction='batchmean'
    ) * (temperature ** 2) 

    hard_loss = F.cross_entropy(student_logits, labels)

    total_loss = alpha * hard_loss + (1 - alpha) * soft_loss
    return total_loss, hard_loss, soft_loss

student_optimizer = AdamW(student_model.parameters(), lr=5e-5)
num_epochs = 3
temperature = 4.0
alpha = 0.3 

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=student_optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

student_model.train()
teacher_model.eval() 

for epoch in range(num_epochs):
    total_train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        student_optimizer.zero_grad()

        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch['labels']

        student_outputs = student_model(**batch)
        student_logits = student_outputs.logits

        with torch.no_grad():
            teacher_outputs = teacher_model(**batch)
            teacher_logits = teacher_outputs.logits

        loss, hard_loss, soft_loss = distillation_loss(
            student_logits, teacher_logits, labels,
            temperature=temperature, alpha=alpha
        )

        total_train_loss += loss.item()

        loss.backward()
        student_optimizer.step()
        lr_scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Ortalama Eğitim Kaybı: {avg_train_loss:.4f}")

student_model.save_pretrained("./distilled_student_model")


Epoch 1/3: 100%|██████████| 1563/1563 [11:28<00:00,  2.27it/s]


Epoch 1 Ortalama Eğitim Kaybı: 2.1306


Epoch 2/3: 100%|██████████| 1563/1563 [11:29<00:00,  2.27it/s]


Epoch 2 Ortalama Eğitim Kaybı: 0.9264


Epoch 3/3: 100%|██████████| 1563/1563 [11:30<00:00,  2.26it/s]


Epoch 3 Ortalama Eğitim Kaybı: 0.6085


In [8]:
baseline_student_model = BertForSequenceClassification(config=student_config)
baseline_student_model.to(device)

baseline_optimizer = AdamW(baseline_student_model.parameters(), lr=5e-5)

baseline_student_model.train()
for epoch in range(num_epochs):
     for batch in tqdm(train_dataloader, desc=f"Baseline Epoch {epoch+1}/{num_epochs}"):
         baseline_optimizer.zero_grad()
         batch = {k: v.to(device) for k, v in batch.items()}
         labels = batch['labels']
         outputs = baseline_student_model(**batch)
         loss = F.cross_entropy(outputs.logits, labels)
         loss.backward()
         baseline_optimizer.step()
baseline_student_model.save_pretrained("./baseline_student_model")

Baseline Epoch 3/3: 100%|██████████| 1563/1563 [05:08<00:00,  5.06it/s]


In [9]:
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import time
import numpy as np

def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    inference_times = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Değerlendirme"):
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch['labels']

            start_time = time.time()
            outputs = model(**batch)
            inference_time = time.time() - start_time
            inference_times.append(inference_time / len(labels)) 

            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary') # veya 'macro'/'weighted'
    avg_inference_time = np.mean(inference_times)

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "avg_inference_time_per_sample": avg_inference_time,
        "total_params": model.num_parameters()
    }

distilled_student_results = evaluate_model(student_model, eval_dataloader, device)
baseline_student_results = evaluate_model(baseline_student_model, eval_dataloader, device)

print("\n--- Damıtılmış Öğrenci Model Sonuçları ---")
print(distilled_student_results)
print("\n--- Baseline Öğrenci Model Sonuçları ---")
print(baseline_student_results)

Değerlendirme: 100%|██████████| 1563/1563 [01:07<00:00, 23.11it/s]


--- Damıtılmış Öğrenci Model Sonuçları ---
{'accuracy': 0.87836, 'f1': 0.8741672528654777, 'precision': 0.9053741321676524, 'recall': 0.84504, 'avg_inference_time_per_sample': 0.00022066958958875347, 'total_params': 28764674}

--- Baseline Öğrenci Model Sonuçları ---
{'accuracy': 0.86436, 'f1': 0.8610530628969474, 'precision': 0.8825703485930282, 'recall': 0.84056, 'avg_inference_time_per_sample': 0.00022055569056585975, 'total_params': 28764674}
